# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [292]:
import pandas as pd
import numpy as np
import requests
import json
import xlsxwriter
import math
from scipy import stats
from secrets import IEX_CLOUD_API_TOKEN

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [293]:
stocks=pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV



## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [294]:
symbol='AAPL'
url=f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data=requests.get(url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [295]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [296]:
final_df=pd.DataFrame(columns=my_columns)
final_df
for symbol_string in symbol_strings:
    url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    
    r=requests.get(url).json()
    
    for stock in r.keys():
        
        df=pd.DataFrame(np.array([stock,r[stock]['quote']['latestPrice'],r[stock]['stats']['year1ChangePercent'],'N/A']).reshape(1,-1),columns=my_columns)
        final_df=pd.concat([final_df,df], ignore_index=True)

final_df.head()   

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,114.57,-0.22226255878287,N/A
1,AAL,13.45,-0.4294670079734785,N/A
2,AAP,171.49,-0.11454158831580243,N/A
3,AAPL,132.47,0.014867091894512692,N/A
4,ABBV,139.68,0.2760755997435204,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [297]:
final_df.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_df=final_df[:50]
final_df.head()
final_df.reset_index(inplace=True)
final_df.head()

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,81.45,2.3244388810075884,N/A
1,147,DVN,58.89,1.319874675995639,N/A
2,355,OXY,57.91,1.0181324215956373,N/A
3,315,MRO,24.31,0.974761866264114,N/A
4,42,APA,40.04,0.9337991474314292,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [298]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input("Enter the size of your portfolio:\n")
    try:
        portfolio_size=float(portfolio_size)
    except ValueError:
        print("not a number. please try again")
        portfolio_size=input("Enter the size of your portfolio:\n")
        portfolio_size=float(portfolio_size)

portfolio_input()
portfolio_size=float(portfolio_size)

Enter the size of your portfolio:
10000000


In [299]:
for stock in final_df.index:
    final_df.iloc[stock,4]=math.floor((float(portfolio_size)/50)/float(final_df.iloc[stock,2]))
    
    


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [300]:
hqm_columns= ['Ticker', 'Price', 'Number of Shares to Buy',
              'One-Year Price Return', 'One-Year Return Percentile',
          
              'Six-Month Price Return', 'Six-Month Return Percentile',
              'Three-Month Price Return', 'Three-Month Return Percentile',
              'One-Month Price Return', 'One-Month Return Percentile']


hqm_df=pd.DataFrame(columns=hqm_columns)


for symbol_string in symbol_strings:
    url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    
    r=requests.get(url).json()
    
    for stock in r.keys():
        
        df=pd.DataFrame(np.array([stock,
                                  r[stock]['quote']['latestPrice'],
                                  'N/A',
                                  r[stock]['stats']['year1ChangePercent'],
                                  '0',
                                  r[stock]['stats']['month6ChangePercent'],
                                  '0',
                                  r[stock]['stats']['month3ChangePercent'],
                                  '0',
                                 r[stock]['stats']['month1ChangePercent'],
                                  '0'
                                 ]).reshape(1,-1),columns=hqm_columns,dtype=(str,float))
        hqm_df=pd.concat([hqm_df,df], ignore_index=True)

hqm_df.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,113.65,N/A,-0.22753692081575,0,-0.253814936552563,0,-0.19340647225992755,0,-0.0677598988051455,0
1,AAL,12.98,N/A,-0.4330725947399516,0,-0.2382847419179876,0,-0.23015468110957873,0,-0.24513514202612333,0
2,AAP,171.2,N/A,-0.1142886582737993,0,-0.27640245313822764,0,-0.18827176441408802,0,-0.13779184456456836,0
3,AAPL,134.37,N/A,0.014843380655652534,0,-0.235128885482391,0,-0.1994971001503691,0,-0.06750865563938888,0
4,ABBV,139.92,N/A,0.275615909240867,0,0.0886548689206967,0,-0.1251118085563392,0,-0.0959395394263112,0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [323]:
hqm_df.Price=pd.to_numeric(hqm_df['Price'])

for column in hqm_columns[3:]:
    hqm_df[column]=pd.to_numeric(hqm_df[column])
    
    
    
time_periods=[
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        change_col=f'{time_period} Price Return'
        percentile_col= f'{time_period} Return Percentile'
        hqm_df.loc[row,percentile_col]=(stats.percentileofscore(hqm_df[change_col],
                                                                                  hqm_df.loc[row,change_col]))/100

        

hqm_df.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
273,LB,83.07,2407608,2.332729,0.992016,0.840418,0.990020,0.215840,0.984032,0.079392,0.970060,98.403194
107,COG,22.87,8745080,0.345036,0.938124,0.185099,0.928144,0.256863,0.986028,0.251933,0.992016,96.107784
470,VRTX,264.74,755458,0.407683,0.948104,0.227979,0.940120,0.057048,0.950100,0.043204,0.946108,94.610778
225,HRB,34.00,5882352,0.490616,0.958084,0.447275,0.974052,0.257519,0.988024,-0.002403,0.864271,94.610778
138,DLTR,157.42,1270486,0.514419,0.962076,0.104069,0.896208,-0.045298,0.820359,0.125415,0.980040,91.467066


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [324]:
from statistics import mean

hqm_df['HQM Score']=0

for row in hqm_df.index:
    l=[]
    for time_period in time_periods:
        
    
        l.append(hqm_df.loc[row,f'{time_period} Return Percentile'])
    hqm_df.loc[row,'HQM Score']=mean(l)

hqm_df


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
273,LB,83.070,2407608,2.332729,0.992016,0.840418,0.990020,0.215840,0.984032,0.079392,0.970060,0.984032
107,COG,22.870,8745080,0.345036,0.938124,0.185099,0.928144,0.256863,0.986028,0.251933,0.992016,0.961078
470,VRTX,264.740,755458,0.407683,0.948104,0.227979,0.940120,0.057048,0.950100,0.043204,0.946108,0.946108
225,HRB,34.000,5882352,0.490616,0.958084,0.447275,0.974052,0.257519,0.988024,-0.002403,0.864271,0.946108
138,DLTR,157.420,1270486,0.514419,0.962076,0.104069,0.896208,-0.045298,0.820359,0.125415,0.980040,0.914671
...,...,...,...,...,...,...,...,...,...,...,...,...
84,CCL,9.900,N/A,-0.663481,0.007984,-0.486596,0.021956,-0.515791,0.005988,-0.317033,0.003992,0.009980
118,CTL,11.000,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,ETFC,50.960,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324,MYL,16.196,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [325]:
hqm_df.sort_values('HQM Score',ascending=False,inplace=True)
hqm_df_final=hqm_df[:50]
hqm_df_final.reset_index(inplace=True, drop=True)
hqm_df_final

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.07,2407608,2.332729,0.992016,0.840418,0.990020,0.215840,0.984032,0.079392,0.970060,0.984032
1,COG,22.87,8745080,0.345036,0.938124,0.185099,0.928144,0.256863,0.986028,0.251933,0.992016,0.961078
2,VRTX,264.74,755458,0.407683,0.948104,0.227979,0.940120,0.057048,0.950100,0.043204,0.946108,0.946108
3,HRB,34.00,5882352,0.490616,0.958084,0.447275,0.974052,0.257519,0.988024,-0.002403,0.864271,0.946108
4,DLTR,157.42,1270486,0.514419,0.962076,0.104069,0.896208,-0.045298,0.820359,0.125415,0.980040,0.914671
5,WRB,68.75,2909090,0.403393,0.946108,0.258297,0.944112,0.045759,0.942116,-0.016075,0.826347,0.914671
6,APA,39.62,5047955,0.926933,0.984032,0.621982,0.986028,0.005915,0.904192,-0.031300,0.764471,0.909681
7,ULTA,412.07,485354,0.212519,0.900200,0.040492,0.830339,0.012166,0.912176,0.135281,0.984032,0.906687
8,AZO,2022.31,98896,0.457298,0.954092,-0.001949,0.768463,0.020889,0.926148,0.085021,0.972056,0.905190
9,VAR,180.77,1106378,0.593197,0.974052,0.030098,0.818363,0.010633,0.908184,0.004952,0.900200,0.900200


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [326]:
portfolio_size=10000000000  #10 billion 
# position_size=(portfolio_size/50)/


for row in hqm_df_final.index:
    hqm_df_final.loc[row,'Number of Shares to Buy']=math.floor((portfolio_size/50)/hqm_df_final.loc[row,'Price'])
    
hqm_df_final

C:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.07,2407608,2.332729,0.992016,0.840418,0.990020,0.215840,0.984032,0.079392,0.970060,0.984032
1,COG,22.87,8745080,0.345036,0.938124,0.185099,0.928144,0.256863,0.986028,0.251933,0.992016,0.961078
2,VRTX,264.74,755458,0.407683,0.948104,0.227979,0.940120,0.057048,0.950100,0.043204,0.946108,0.946108
3,HRB,34.00,5882352,0.490616,0.958084,0.447275,0.974052,0.257519,0.988024,-0.002403,0.864271,0.946108
4,DLTR,157.42,1270486,0.514419,0.962076,0.104069,0.896208,-0.045298,0.820359,0.125415,0.980040,0.914671
5,WRB,68.75,2909090,0.403393,0.946108,0.258297,0.944112,0.045759,0.942116,-0.016075,0.826347,0.914671
6,APA,39.62,5047955,0.926933,0.984032,0.621982,0.986028,0.005915,0.904192,-0.031300,0.764471,0.909681
7,ULTA,412.07,485354,0.212519,0.900200,0.040492,0.830339,0.012166,0.912176,0.135281,0.984032,0.906687
8,AZO,2022.31,98896,0.457298,0.954092,-0.001949,0.768463,0.020889,0.926148,0.085021,0.972056,0.905190
9,VAR,180.77,1106378,0.593197,0.974052,0.030098,0.818363,0.010633,0.908184,0.004952,0.900200,0.900200


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [327]:
writer=pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_df_final.to_excel(writer,sheet_name="Momentum Strategy",index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [328]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [332]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for col in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{col}:{col}',22,column_formats[col][1])
    writer.sheets['Momentum Strategy'].write(f'{col}1',column_formats[col][0],column_formats[col][1])
    
    
writer.save()

C:\anaconda\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
